In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv("C:\\Users\\anujs\\OneDrive\\Desktop\\datasets\\cic_iomt.csv")

In [5]:
X = df.drop(columns=['Attack', 'Category', 'Binary'])
y = df['Binary']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)



In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [10]:
def dirichlet_split(X, y, num_clients=5, alpha=0.3):
    classes = np.unique(y)
    client_indices = [[] for _ in range(num_clients)]

    for c in classes:
        idx_c = np.where(y == c)[0]
        np.random.shuffle(idx_c)

        proportions = np.random.dirichlet(alpha * np.ones(num_clients))
        proportions = np.cumsum(proportions)
        proportions = (proportions * len(idx_c)).astype(int)[:-1]

        split_indices = np.split(idx_c, proportions)

        for i in range(num_clients):
            client_indices[i].extend(split_indices[i])

    clients = [(X.iloc[idx].values, y.iloc[idx].values) for idx in client_indices]
    return clients
clients = dirichlet_split(X, y, num_clients=5, alpha=0.3)


In [11]:
for i, (_, y_client) in enumerate(clients):
    unique, counts = np.unique(y_client, return_counts=True)
    print(f"Client {i} distribution:", dict(zip(unique, counts)))


Client 0 distribution: {'Attack': 2160201, 'Benign': 56167}
Client 1 distribution: {'Attack': 436131, 'Benign': 141}
Client 2 distribution: {'Attack': 1361852, 'Benign': 28}
Client 3 distribution: {'Attack': 61584, 'Benign': 49806}
Client 4 distribution: {'Attack': 4513896, 'Benign': 124197}


In [12]:
from torch.utils.data import TensorDataset, DataLoader

def get_dataloader(X, y, batch_size=64):
    dataset = TensorDataset(
        torch.tensor(X, dtype=torch.float32),
        torch.tensor(y, dtype=torch.long)
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)
